In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string
import re
import itertools
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
#ICD代码和含义
D_ICD_DIAG = pd.read_csv('./mimic/D_ICD_DIAGNOSES.csv.gz',compression='gzip')
D_ICD_PROC = pd.read_csv('./mimic/D_ICD_PROCEDURES.csv.gz',compression='gzip')

In [6]:
#ICD代码原始文件
diagnoses_icd = pd.read_csv('./mimic/DIAGNOSES_ICD.csv.gz',compression='gzip')
procedures_icd = pd.read_csv('./mimic/PROCEDURES_ICD.csv.gz',compression='gzip')

In [11]:
diagnoses_icd

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [8]:
#读取notes源文件
NOTES = pd.read_csv('./mimic/NOTEEVENTS.csv.gz', compression='gzip')
KEEP = NOTES[['HADM_ID','CATEGORY','TEXT']]

In [44]:
len(diagnoses_icd['ICD9_CODE'].unique())

6985

In [ ]:
# SAMPLE_20K = pd.read_csv('./Preprocess/SAMPLES.csv')

In [9]:
KEEP

,HADM_ID,CATEGORY,TEXT
0,167853.0,Discharge summary,Admission Date: [**2151-7-16**] Dischar...
1,107527.0,Discharge summary,Admission Date: [**2118-6-2**] Discharg...
2,167118.0,Discharge summary,Admission Date: [**2119-5-4**] D...
3,196489.0,Discharge summary,Admission Date: [**2124-7-21**] ...
4,135453.0,Discharge summary,Admission Date: [**2162-3-3**] D...
...,...,...,...
2083175,115637.0,Nursing/other,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,115637.0,Nursing/other,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,115637.0,Nursing/other,Family Meeting Note\nFamily meeting held with ...
2083178,115637.0,Nursing/other,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [26]:
KEEP = KEEP.groupby(['HADM_ID']).agg({'TEXT': ' '.join, 'CATEGORY': ' '.join})

In [14]:
len(KEEP['HADM_ID'].unique())

58362

In [21]:
diagnoses_dict = {}
for i in range(len(diagnoses_icd)):
    entry = diagnoses_icd.iloc[i]
    hadm = entry['HADM_ID']
    icd = entry['ICD9_CODE']
    if hadm not in diagnoses_dict:
        diagnoses_dict[hadm] = [icd]
    else:
        diagnoses_dict[hadm].append(icd)
        
procedures_dict = {}
for i in range(len(procedures_icd)):
    entry = procedures_icd.iloc[i]
    hadm = entry['HADM_ID']
    icd = entry['ICD9_CODE']
    if hadm not in procedures_dict:
        procedures_dict[hadm] = [icd]
    else:
        procedures_dict[hadm].append(icd)

In [22]:
diagnoses_df = pd.DataFrame.from_dict(diagnoses_dict,orient='index')
procedures_df = pd.DataFrame.from_dict(procedures_dict,orient='index')

In [36]:
diagnoses_df

,DIAG_CODE1,DIAG_CODE2,DIAG_CODE3,DIAG_CODE4,DIAG_CODE5,DIAG_CODE6,DIAG_CODE7,DIAG_CODE8,DIAG_CODE9,DIAG_CODE10,...,DIAG_CODE31,DIAG_CODE32,DIAG_CODE33,DIAG_CODE34,DIAG_CODE35,DIAG_CODE36,DIAG_CODE37,DIAG_CODE38,DIAG_CODE39,DIAG_CODES
HADM_ID,,,,,,,,,,,,,,,,,,,,,
172335,40301,486,58281,5855,4254,2762,7100,2767,7243,45829,...,None,None,None,None,None,None,None,None,None,"40301,486,58281,5855,4254,2762,7100,2767,7243,..."
173633,40301,5856,58381,7100,5589,2875,28521,45829,32723,22804,...,None,None,None,None,None,None,None,None,None,"40301,5856,58381,7100,5589,2875,28521,45829,32..."
174105,53100,41071,2859,41401,725,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"53100,41071,2859,41401,725"
109976,1915,3314,53081,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"1915,3314,53081"
178393,41401,4111,48283,2859,2720,3051,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"41401,4111,48283,2859,2720,3051"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172304,20280,3485,7843,4019,2720,53081,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"20280,3485,7843,4019,2720,53081"
152542,566,25062,3572,V5867,42731,4019,4280,53081,3051,27800,...,None,None,None,None,None,None,None,None,None,"566,25062,3572,V5867,42731,4019,4280,53081,305..."
161999,43411,3485,3484,430,34830,99731,51883,5990,34291,29181,...,None,None,None,None,None,None,None,None,None,"43411,3485,3484,430,34830,99731,51883,5990,342..."


In [23]:
diagnoses_df.columns = ['DIAG_CODE'+str(i) for i in range(1,len(diagnoses_df.columns)+1)]
diagnoses_df.index.name = 'HADM_ID'
procedures_df.columns = ['PRCD_CODE'+str(i) for i in range(1,len(procedures_df.columns)+1)]
procedures_df.index.name = 'HADM_ID'
codes_df = pd.merge(diagnoses_df, procedures_df, how='outer', on='HADM_ID')

In [24]:
diagnoses_df['DIAG_CODES'] = diagnoses_df[diagnoses_df.columns[:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

procedures_df['PROC_CODES'] = procedures_df[procedures_df.columns[:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [25]:
diagnoses = diagnoses_df[['DIAG_CODES']]
procedures = procedures_df[['PROC_CODES']]
codes = pd.merge(diagnoses, procedures, how='outer', on='HADM_ID')
codes = codes.dropna()

In [27]:
codes.to_csv('CODES.csv')

In [30]:
# KEEP.set_index('HADM_ID')
merged_df = pd.merge(KEEP, codes, how='left', on='HADM_ID')
merged_df = merged_df.dropna()

In [31]:
merged_df

,TEXT,CATEGORY,DIAG_CODES,PROC_CODES
HADM_ID,,,,
100003.0,Admission Date: [**2150-4-17**] ...,Discharge summary Echo ECG Nursing Physician ...,"53100,2851,07054,5715,45621,53789,4019,53550,7823","4443.0,9607.0,9904.0,3893.0"
100006.0,Admission Date: [**2108-4-6**] Discharg...,Discharge summary Discharge summary Echo ECG R...,"49320,51881,486,20300,2761,7850,3090,V1251,V1582","9390.0,9925.0"
100007.0,Admission Date: [**2145-3-31**] ...,Discharge summary ECG Nursing/other Nursing/ot...,"56081,5570,9973,486,4019","4562.0,5459.0"
100009.0,Admission Date: [**2162-5-16**] ...,Discharge summary Echo ECG Radiology Radiology...,"41401,99604,4142,25000,27800,V8535,4148,4111,V...","3613.0,3615.0,3795.0,3961.0"
100010.0,Admission Date: [**2109-12-10**] ...,Discharge summary Radiology,"1890,1961,1987,1976,27652","5551.0,540.0,403.0"
...,...,...,...,...
199993.0,Admission Date: [**2161-10-23**] Discha...,Discharge summary ECG ECG ECG ECG ECG ECG Radi...,"41031,42821,42731,4271,5180,4240,2760,5119,5184","3614.0,3512.0,3761.0,8842.0,8848.0,3961.0,3964..."
199994.0,Admission Date: [**2188-7-7**] Discharg...,Discharge summary ECG Radiology Radiology Radi...,"486,4280,51881,3970,496,4169,585,42732,2762","9671.0,9604.0,3995.0,3891.0"
199995.0,Admission Date: [**2137-12-11**] Discha...,Discharge summary Echo ECG ECG ECG ECG ECG Rad...,"4210,7464,42971,30401,4412,44284,V1259,04111,3...","3521.0,3961.0,3845.0,3539.0,8841.0,8847.0,9929..."


In [32]:
SAMPLE = merged_df.sample(n=20000)

In [33]:
SAMPLE.to_csv('SAMPLE_20K.csv')

In [34]:
SAMPLE.columns

Index(['TEXT', 'CATEGORY', 'DIAG_CODES', 'PROC_CODES'], dtype='object')

In [35]:
SAMPLE

,TEXT,CATEGORY,DIAG_CODES,PROC_CODES
HADM_ID,,,,
190979.0,Admission Note\n[**Name (NI) 3762**] mom is ve...,Nursing/other Nursing/other Nursing/other Nurs...,"V3000,V502",640.0
187350.0,Admission Date: [**2165-9-27**] ...,Discharge summary Radiology Radiology Radiolog...,"5770,27651,00845,57450,4019,25000","5185.0,5187.0,3893.0"
129078.0,Admission Date: [**2163-8-16**] ...,Discharge summary Echo ECG ECG ECG Radiology R...,"42731,486,1510,45384,27651,7140,V1251,V1582",9962.0
133981.0,Admission Date: [**2167-4-29**] ...,Discharge summary ECG ECG Radiology Radiology ...,"1976,5845,34830,1977,78951,1572,56089,V4986,27...",5491.0
147056.0,Admission Date: [**2155-9-14**] ...,Discharge summary Radiology Radiology Radiolog...,"5712,45620,30500,2875,135,5723,2720,49390","4233.0,9905.0,9904.0"
...,...,...,...,...
152254.0,Admission Date: [**2142-3-11**] ...,Discharge summary Echo ECG ECG ECG Radiology R...,"0389,5070,78552,51881,5307,34982,70724,48241,5...","9671.0,9604.0,3891.0,3893.0,3897.0,14.0,966.0"
112758.0,Admission Date: [**2114-1-16**] Dischar...,Discharge summary Echo ECG Radiology Radiology...,"V3001,769,7742,77211,7755,76516,76526,75614,7470","9915.0,3893.0,9671.0,9604.0,9983.0,966.0"
116726.0,Admission Date: [**2104-9-29**] ...,Discharge summary Discharge summary Echo ECG E...,"3963,41401,39891,40391,5856,4260,42731,25040,V...","3524.0,3611.0,3772.0,3615.0,3783.0,3964.0,3995..."


### Codes to Dict

In [155]:
sample_ids = SAMPLE.index

In [156]:
flt_diag = diagnoses_icd[diagnoses_icd['HADM_ID'].isin(sample_ids)]
flt_proc = procedures_icd[procedures_icd['HADM_ID'].isin(sample_ids)]

In [157]:
diag_keep = flt_diag['ICD9_CODE'].value_counts()[:300]
proc_keep = flt_proc['ICD9_CODE'].value_counts()[:100]

In [158]:
diag2idx, idx2diag = {},{}
for d in diag_keep.index:
    if d not in diag2idx:
        idx2diag[len(idx2diag)] = d
        diag2idx[d] = len(diag2idx)
        
proc2idx, idx2proc = {},{}
for p in proc_keep.index:
    if p not in proc2idx:
        idx2proc[len(idx2proc)] = p
        proc2idx[p] = len(proc2idx)

In [159]:
with open('diag2idx.pickle','wb') as f:
    pickle.dump(diag2idx,f,pickle.HIGHEST_PROTOCOL)
with open('idx2diag.pickle','wb') as f:
    pickle.dump(idx2diag,f,pickle.HIGHEST_PROTOCOL)
with open('proc2idx.pickle','wb') as f:
    pickle.dump(proc2idx,f,pickle.HIGHEST_PROTOCOL)
with open('idx2proc.pickle','wb') as f:
    pickle.dump(idx2proc,f,pickle.HIGHEST_PROTOCOL)

### CONVERT CODE LIST

In [160]:
def diag_code2idx(org_lst):
    coded_lst = []
    for c in org_lst.split(','):
        if c in diag2idx:
            coded_lst.append(diag2idx[c])
    return coded_lst

In [161]:
def proc_code2idx(org_lst):
    coded_lst = []
    for c in org_lst.split(','):
        c_ = int(str(c).split('.')[0])
        if c_ in proc2idx:
            coded_lst.append(proc2idx[c_])
            
    return coded_lst

In [162]:
SAMPLE['CODED_DIAG'] = SAMPLE['DIAG_CODES'].apply(diag_code2idx)
SAMPLE['CODED_PROC'] = SAMPLE['PROC_CODES'].apply(proc_code2idx)

In [163]:
SAMPLE

,HADM_ID,CATEGORY,TEXT,DIAG_CODES,PROC_CODES,CODED_DIAG,CODED_PROC
1214663,197778.0,Radiology,[**2172-5-9**] 4:52 PM\n CHEST (PORTABLE AP) ...,"57142,5856,5762,5722,5845,6824,40391,45621,790...","5059.0,93.0,5011.0,4523.0,5185.0,5187.0,966.0,...","[61, 256, 39, 42, 230, 49, 130, 152, 40, 92, 5...","[84, 54, 89, 55, 2, 28, 28, 0, 2, 17, 1, 3]"
1964832,154221.0,Nursing/other,NPN NIGHTS\n\n\nAlt resp r/t prematurity: Rema...,"V3001,76502,769,7718,7707,7742,7766,7798,36221","5300.0,9604.0,9672.0,9983.0,1424.0,9915.0","[28, 86, 59]","[1, 5, 30, 12]"
1307650,177919.0,Nursing/other,"CSRU PROGRESS NOTE\nS/O: NEURO: Alert, comfort...","41401,4111,42731,4280,78820,4019,25000,2720,V4364","3615.0,3612.0,3961.0","[2, 57, 1, 3, 120, 0, 4, 10, 105]","[10, 39, 7]"
1113204,111516.0,Radiology,[**2141-3-25**] 10:21 PM\n CT CHEST W/CONTRAST...,"80115,3314,8730,E8809,4019,2720,3320","966.0,9671.0","[150, 0, 10, 142]","[2, 3]"
1975873,196570.0,Nursing/other,Neonatology Attending Note\nProcedure Note:\n\...,"V3101,76503,769,7742,7793,76524,77981,77082,77...","9671.0,9604.0,9390.0,966.0,3893.0,9915.0,9983....","[180, 86, 59, 140, 285, 11, 9]","[3, 1, 20, 2, 0, 12, 30, 8]"
...,...,...,...,...,...,...,...
13573,161829.0,Discharge summary,Admission Date: [**2102-7-17**] ...,"4241,4254,4280,20190,9971,42731,2449,53081","3521.0,3961.0","[81, 79, 3, 54, 1, 17, 13]","[49, 7]"
1777276,152050.0,Nursing/other,Nursing Progress Note\n\n\n#1- O/A: Infant rec...,"V3001,769,7742,76515,76524,77081,36221,7470,V290","9672.0,9604.0,9390.0,9983.0,3893.0","[28, 86, 59, 114, 294, 11]","[5, 1, 20, 30, 0]"
156919,103464.0,ECG,Sinus bradycardia. Left atrial abnormality. T ...,"41401,4111,4280,25060,3572,4019,4168,2720,7812","3613.0,3723.0,3615.0,3961.0,8856.0","[2, 57, 3, 92, 53, 0, 55, 10]","[22, 11, 10, 7, 6]"
1847276,135902.0,Nursing/other,Neonatology - NNP Progress Note\n\n[**Known la...,"V3000,76503,769,7707,7742,2880,76075,7470,V290","9604.0,9672.0,9915.0,966.0,3893.0,3891.0,9983.0","[23, 86, 59, 294, 11]","[1, 5, 12, 2, 0, 9, 30]"


### 常规处理

In [ ]:
def remove_stopwords(text): 
        stop_words = set(stopwords.words("english")) 
        word_tokens = word_tokenize(text) 
        filtered_text = [word for word in word_tokens if word not in stop_words] 
        return filtered_text 
    
def preprocess(note):
    note = note.replace('\n',' ')
    note = note.replace('w/', 'with')
    note = note.lower() #lower case
    note = re.sub(r'\d+', '', note) #remove numbers
    note = note.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    note = " ".join(note.split())
    note = remove_stopwords(note)
    return note

In [ ]:
sample_1k_removed['NOTE'] = sample_1k_removed['TEXT'].apply(preprocess)
sample_10k_removed['NOTE'] = sample_10k_removed['TEXT'].apply(preprocess)
merged_df_removed['NOTE'] = merged_df_removed['TEXT'].apply(preprocess)

In [ ]:
# sample_1k_cleaned = sample_1k_removed[['NOTE','CODED_DIAG','CODED_PROC']]
# sample_10k_cleaned = sample_10k_removed[['NOTE','CODED_DIAG','CODED_PROC']]
# merged_df_cleaned = merged_df_removed[['NOTE','CODED_DIAG','CODED_PROC']]

In [ ]:
sample_1k_removed['CODED_NOTE'] = sample_1k_cleaned['NOTE']
sample_10k_removed['CODED_NOTE'] = sample_10k_cleaned['NOTE']
merged_df_removed['CODED_NOTE'] = merged_df_cleaned['NOTE']

In [ ]:
sample_10k_removed.to_csv('SAMPLE10K_ALL.csv')
sample_1k_removed.to_csv('SAMPLE1K_ALL.csv')
merged_df_removed.to_csv('ALL.csv')

In [ ]:
sample_20k = merged_df_removed.sample(n = 20000) 
sample_20k.to_csv('SAMPLE20K_ALL.csv')

In [ ]:
sample_20k

### corpus

In [169]:
TOKENS = pd.read_csv('19908_all_coded_token.csv')

In [170]:
TOKENS

,Unnamed: 0,HADM_ID,CATEGORY,TEXT,HEADERS,padded_tokens
0,0,197778.0,Radiology,4:52 PM CHEST (PORTABLE AP) ...,[],"['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', ..."
1,1,154221.0,Nursing/other,NPN NIGHTS\nAlt resp r/t prematurity: Remains ...,"['FEN', 'Alt in Growth and Development']","['FEN', 'Alt', 'in', 'Growth', 'and', 'Develop..."
2,2,177919.0,Nursing/other,"CSRU PROGRESS NOTE\n CV: AFIB up to 130, r...",['S/O'],"['S/O', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<..."
3,3,111516.0,Radiology,10:21 PM CT CHEST W/CONTRAST; CT ABDOMEN W/CO...,['IMPRESSION'],"['IMPRESSION', '<PAD>', '<PAD>', '<PAD>', '<PA..."
4,4,196570.0,Nursing/other,Neonatology Attending Note\nBased on the last ...,"['Procedure Note', 'Procedure']","['Procedure', 'Note', 'Procedure', '<PAD>', '<..."
...,...,...,...,...,...,...
19903,19995,161829.0,Discharge summary,Sex: F\nCARDIOTHORACIC\nAllergies: Bleomycin /...,"['History of Present Illness', 'Discharge Diag...","['History', 'of', 'Present', 'Illness', 'Disch..."
19904,19996,152050.0,Nursing/other,Nursing Progress Note\n#1- O/A: Infant receive...,"[' #4- O/A', '#5- O/A']","['#4-', 'O/A', '#5-', 'O/A', '<PAD>', '<PAD>',..."
19905,19997,103464.0,ECG,Sinus bradycardia.\nLeft atrial abnormality.\n...,[],"['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', ..."
19906,19998,135902.0,Nursing/other,Neonatology -\nNNP Progress Note\nis active wi...,[],"['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', ..."


In [ ]:
corpus = {}
for i, s in enumerate(SAMPLE['NOTE']):
    for w in s:
        corpus[w] = corpus.get(w, 1) + 1
corpus = {k: v for k, v in sorted(corpus.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
corpus_slice = dict(itertools.islice(corpus.items(), 10000))

In [ ]:
# build dictionary
word2idx = {'<PAD>': 0, '<UNK>':1}
idx2word = {0: '<PAD>', 1:'<UNK>'}
for c in corpus_slice:
    word2idx[c] = len(word2idx)
    idx2word[len(idx2word)] = c

In [ ]:
def note2idx(org_lst):
    coded_lst = []
    for w in org_lst:
        if w in word2idx:
            coded_lst.append(word2idx[w])
        else:
            coded_lst.append(0)
    return coded_lst

In [ ]:
def note2idx_cap400(org_lst):
    coded_lst = []
    for w in org_lst:
        if len(coded_lst) < 400 and w in word2idx:
            coded_lst.append(word2idx[w])
        else:
            coded_lst.append(1)
    coded_lst += [0]*(400-len(coded_lst))
    return coded_lst

In [ ]:
sample_20k['CODED_NOTE'] = sample_20k['NOTE'].apply(note2idx_cap400)

In [ ]:
sample_20k.to_csv('SAMPLE_2OK.csv')

In [ ]:
# sample_1k_cleaned['NOTE'] = sample_1k_cleaned['NOTE'].apply(note2idx)
# sample_10k_cleaned['NOTE'] = sample_10k_cleaned['NOTE'].apply(note2idx)
# merged_df_cleaned['NOTE'] = merged_df_cleaned['NOTE'].apply(note2idx)

In [ ]:
# sample_10k_cleaned.to_csv('SAMPLE10K.csv')
# sample_1k_cleaned.to_csv('SAMPLE1K.csv')
# merged_df_cleaned.to_csv('CLEANED.csv')

In [ ]:
dump_lst = [diag2idx, idx2diag, proc2idx, idx2proc, word2idx, idx2word]
with open('diag2idx.pickle','wb') as f:
    pickle.dump(diag2idx,f,pickle.HIGHEST_PROTOCOL)
with open('idx2diag.pickle','wb') as f:
    pickle.dump(idx2diag,f,pickle.HIGHEST_PROTOCOL)
with open('proc2idx.pickle','wb') as f:
    pickle.dump(proc2idx,f,pickle.HIGHEST_PROTOCOL)
with open('idx2proc.pickle','wb') as f:
    pickle.dump(idx2proc,f,pickle.HIGHEST_PROTOCOL)
with open('word2idx.pickle','wb') as f:
    pickle.dump(word2idx,f,pickle.HIGHEST_PROTOCOL)
with open('idx2word.pickle','wb') as f:
    pickle.dump(idx2word,f,pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('corpus.pickle','wb') as f:
    pickle.dump(corpus,f,pickle.HIGHEST_PROTOCOL)

#################################################################

# Codes

In [56]:
diagnoses_icd

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [ ]:
# diagnoses_icd[diagnoses_icd['HADM_ID']==172335]

In [9]:
diagnoses_dict = {}
for i in range(len(diagnoses_icd)):
    entry = diagnoses_icd.iloc[i]
    hadm = entry['HADM_ID']
    icd = entry['ICD9_CODE']
    if hadm not in diagnoses_dict:
        diagnoses_dict[hadm] = [icd]
    else:
        diagnoses_dict[hadm].append(icd)
        
procedures_dict = {}
for i in range(len(procedures_icd)):
    entry = procedures_icd.iloc[i]
    hadm = entry['HADM_ID']
    icd = entry['ICD9_CODE']
    if hadm not in procedures_dict:
        procedures_dict[hadm] = [icd]
    else:
        procedures_dict[hadm].append(icd)

In [13]:
diagnoses_df = pd.DataFrame.from_dict(diagnoses_dict,orient='index')
procedures_df = pd.DataFrame.from_dict(procedures_dict,orient='index')

In [14]:
diagnoses_df.columns = ['DIAG_CODE'+str(i) for i in range(1,len(diagnoses_df.columns)+1)]
diagnoses_df.index.name = 'HADM_ID'
procedures_df.columns = ['PRCD_CODE'+str(i) for i in range(1,len(procedures_df.columns)+1)]
procedures_df.index.name = 'HADM_ID'
codes_df = pd.merge(diagnoses_df, procedures_df, how='outer', on='HADM_ID')

In [16]:
diagnoses_df['DIAG_CODES'] = diagnoses_df[diagnoses_df.columns[:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

procedures_df['PROC_CODES'] = procedures_df[procedures_df.columns[:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [23]:
diagnoses = diagnoses_df[['DIAG_CODES']]
procedures = procedures_df[['PROC_CODES']]
codes = pd.merge(diagnoses, procedures, how='outer', on='HADM_ID')

In [ ]:
diagnoses_df.to_csv('DIAGNOSES_DF.csv')
procedures_df.to_csv('PROCEDURES_DF.csv')
codes_df.to_csv('CODES_DF.csv')


In [26]:
codes.to_csv('CODES.csv')

# Notes

In [31]:
notes = pd.read_csv('./mimic/NOTEEVENTS.csv.gz', compression='gzip')
notes_df = notes[['HADM_ID','TEXT']]
notes_df.set_index('HADM_ID')
merged_df = pd.merge(notes_df, codes, how='left', on='HADM_ID')
merged_df = merged_df.dropna()
merged_df.to_csv('FULL_DATA.csv')

,TEXT
HADM_ID,
167853.0,Admission Date: [**2151-7-16**] Dischar...
107527.0,Admission Date: [**2118-6-2**] Discharg...
167118.0,Admission Date: [**2119-5-4**] D...
196489.0,Admission Date: [**2124-7-21**] ...
135453.0,Admission Date: [**2162-3-3**] D...
...,...
115637.0,NPN\n\n\n#1 Infant remains in RA with O2 sats...
115637.0,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
115637.0,Family Meeting Note\nFamily meeting held with ...


In [ ]:
### SLICE NOTE
sample_1k = merged_df.sample(n = 1000) 
sample_1k.to_csv('SAMPLE1K.csv')
sample_10k = merged_df.sample(n = 10000) 
sample_10k.to_csv('SAMPLE10K.csv')

In [35]:
notes.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT'],
      dtype='object')

In [44]:
merged_df.columns

Index(['HADM_ID', 'TEXT', 'DIAG_CODES', 'PROC_CODES'], dtype='object')

In [38]:
notes['CATEGORY'].unique()

array(['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ',
       'Rehab Services', 'Case Management ', 'Respiratory ', 'Nutrition',
       'General', 'Social Work', 'Pharmacy', 'Consult', 'Radiology',
       'Nursing/other'], dtype=object)

In [64]:


a = 'test a b c'
b = ('aaa', 'bbb', 'test')


In [65]:
a.startswith(b)

True

In [39]:
# 去deidentify信息
full_data['TEXT'] = full_data['TEXT'].replace(to_replace=r"\[.*?\]", value="", regex=True)